In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

True

In [3]:
from memory import *

/usr/local/anaconda3/envs/selfenv/lib/python3.8/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
chunk_size: int = 400
chunk_overlap: int = 50
dimension: int = 1536
model_name: str = "gpt-3.5-turbo"
temperature: float = 0.5

In [31]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def find_tokens(tokenizer: Encoding, text: str) -> Tuple[List[int], int]:
    tokens: List[int] = tokenizer.encode(text, disallowed_special=())
    return tokens, len(tokens)

tokenizer = tiktoken.get_encoding("cl100k_base")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=lambda text: find_tokens(tokenizer, text)[1],
    separators=SEPERATORS,
)
embedder = OpenAIEmbeddings(
    model=EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY,
    show_progress_bar=True,
)
(
    _db_vector_store,
    db_search_tool,
    db_store_tool,
) = pinecone_tool(
    index=PINECONE_INDEX_NAME,
    text_splitter=text_splitter,
    embedder=embedder,
    dimension=dimension,
)

# NOTE: that this model and agent is not syncronous
# despite setting streamign to true
open_ai_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name=model_name,
    temperature=temperature,
    streaming=True,
    callbacks=[FinalStreamingStdOutCallbackHandler(answer_prefix_tokens=['output'])],
)

search_tool = search_api_tool()
other_tools = load_tools(["wikipedia"])
tools = [
    db_search_tool,
    db_store_tool,
    search_tool,
    *other_tools,
]
convo_openai_agent = create_conversational_retrieval_agent(
    llm=open_ai_llm,
    tools=tools,
    verbose=False,
)

In [27]:
convo_openai_agent({"input": "Create a 5 page story of an octopus and a giraffe falling in love"})

{'input': 'Create a 5 page story of an octopus and a girraffe falling in love',
 'chat_history': [HumanMessage(content='Create a 5 page story of an octopus and a girraffe falling in love', additional_kwargs={}, example=False),
  AIMessage(content="Once upon a time, in a deep blue ocean, there lived a curious octopus named Oliver. Oliver loved exploring the vast underwater world and meeting new creatures. One day, while swimming near the surface, he spotted a beautiful giraffe named Grace, who had wandered too far from her home on the savannah.\n\nIntrigued by Grace's long neck and graceful movements, Oliver couldn't help but be drawn to her. He mustered up the courage to approach her and struck up a conversation. They quickly discovered that despite their differences, they shared a love for adventure and a curiosity about the world.\n\nAs their friendship grew, Oliver and Grace spent their days exploring the ocean depths and the land above. They discovered hidden treasures in sunken sh

In [32]:
convo_openai_agent({"input": "Create a 2 sentence story of an octopus and a girraffe falling in love"})

{'input': 'Create a 2 sentence story of an octopus and a girraffe falling in love',
 'chat_history': [HumanMessage(content='Create a 2 sentence story of an octopus and a girraffe falling in love', additional_kwargs={}, example=False),
  AIMessage(content='In a whimsical underwater world, an adventurous octopus and a graceful giraffe met by chance and their hearts entwined. Despite their differences in habitat, their love transcended boundaries, inspiring a magical dance of love that captivated the entire animal kingdom.', additional_kwargs={}, example=False)],
 'output': 'In a whimsical underwater world, an adventurous octopus and a graceful giraffe met by chance and their hearts entwined. Despite their differences in habitat, their love transcended boundaries, inspiring a magical dance of love that captivated the entire animal kingdom.',
 'intermediate_steps': []}